In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.metrics import pairwise_distances
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

data_path = "./dataset/datasetTC4.dat"

data = np.loadtxt(data_path)
print(data.shape)
data

(310, 6)


array([[ 63.03,  22.55,  39.61,  40.48,  98.67,  -0.25],
       [ 39.06,  10.06,  25.02,  29.  , 114.41,   4.56],
       [ 68.83,  22.22,  50.09,  46.61, 105.99,  -3.53],
       ...,
       [ 61.45,  22.69,  46.17,  38.75, 125.67,  -2.71],
       [ 45.25,   8.69,  41.58,  36.56, 118.55,   0.21],
       [ 33.84,   5.07,  36.64,  28.77, 123.95,  -0.2 ]])

In [66]:
# Normalização do dataset usando z-score
def z_score_norm(dataset):
    std = np.std(dataset, axis=0)
    normalized = (dataset - np.mean(dataset, axis=0)) / np.where(std == 0, 1e-9, std) # Caso divisão por zero
    return normalized

data = z_score_norm(data)
data

array([[ 0.14722652,  0.50111133, -0.66512805, -0.18460234, -1.44783071,
        -0.70794606],
       [-1.24570706, -0.74889057, -1.45276272, -1.04124965, -0.26402779,
        -0.57967342],
       [ 0.48427345,  0.46808485, -0.0993699 ,  0.27282344, -0.89729467,
        -0.79541679],
       ...,
       [ 0.05541029,  0.51512256, -0.31098936, -0.31369641,  0.58283504,
        -0.77354911],
       [-0.88599664, -0.88600047, -0.55877847, -0.47711606,  0.04734096,
        -0.69567882],
       [-1.54904929, -1.24829085, -0.82546218, -1.05841244,  0.45347411,
        -0.70661266]])

In [67]:
# Inicialização dos parâmetros
Kmax = 10 # Número de protótipos
rounds = 30 # Número de rodadas
epochs = 20 # Número de iterações
ref_index = "Dunn" # Índice de referência para escolha do K

In [68]:
# Implementação do Índice de Dunn, oss outros índices foram usados do Scikit-learn
def dunn_index(data, labels):
    min_intercluster_dists = np.max(pairwise_distances(data, metric='euclidean'))
    max_intracluster_dists = 0.0
    
    for cluster_label in np.unique(labels):
        cluster_points = data[labels == cluster_label]
        
        if cluster_label < len(np.unique(labels)) - 1:
            not_cluster_points = data[labels > cluster_label]
            intercluster_dists = pairwise_distances(cluster_points, not_cluster_points, metric='euclidean')
            min_intercluster_dists = min(np.min(intercluster_dists), min_intercluster_dists)
            
        intracluster_dists = pairwise_distances(cluster_points, metric='euclidean')
        max_intracluster_dists = max(np.max(intracluster_dists), max_intracluster_dists)

    return min_intercluster_dists / max_intracluster_dists

In [69]:
# Batch K-means
def kmeans(data, proto, epochs):
    N, p = data.shape
    K = proto.shape[0]
    SSD = np.zeros(epochs)

    updt_proto = np.copy(proto)
    
    for epoch in range(epochs):
        dist = np.zeros((N, K))
        for t in range(N):
            dist[t, :] = np.linalg.norm(data[t, :] - updt_proto, axis=1)
        
        cluster = np.argmin(dist, axis=1)
        SSD[epoch] = np.sum(np.min(dist, axis=1)**2)

        for k in range(K):
            I = np.where(cluster == k)[0]
            partition = data[I, :]
            updt_proto[k, :] = np.mean(partition, axis=0)

    return updt_proto, SSD

In [70]:
# Rodada prévia para encontrar o melhor valor de K por cada índice
k_by_dunn, k_by_ch, k_by_db = [], [], []

for ext_rnd in range(100):
    best_dunns, best_chs, best_dbs = [], [], []
    
    for K in range(2, Kmax+1):
        rnd_final_prototypes, rnd_ssds = [], []
        
        for rnd in range(10): # Aplicando K-médias nos dados por rodada
            idxs = np.random.choice(data.shape[0], K, replace=False) # Aleatoriedade dos protótipos
            prototypes = data[idxs, :] # Protótipos definidos
            
            prototypes, kmeans_SSD = kmeans(data, prototypes, epochs) # Aplicando o K-means
            
            rnd_final_prototypes.append(prototypes)
            rnd_ssds.append(kmeans_SSD)
        
        k_best_ssd_idx = np.argmin([ssd[-1] for ssd in rnd_ssds]) # Pega o índice da rodada com menor SSD final
        k_best_final_prototypes = rnd_final_prototypes[k_best_ssd_idx]
    
        labels = np.argmin(cdist(data, k_best_final_prototypes), axis=1) # Cálcula as labels (clusters) de cada amostra
    
        best_dunns.append(dunn_index(data, labels)) # Salvando o Índice de Dunn de K
        best_chs.append(calinski_harabasz_score(data, labels)) # Salvando o Índice de Calinski-Harabasz de K
        best_dbs.append(davies_bouldin_score(data, labels)) # Salvando o Índice de Davies-Bouldin de K
    
    k_by_dunn.append(np.argmax(best_dunns) + 2) # Melhor K segundo o Índice de Dunn
    k_by_ch.append(np.argmax(best_chs) + 2) # Melhor K segundo o Índice de Calinski-Harabasz
    k_by_db.append(np.argmin(best_dbs) + 2) # Melhor K segundo o Índice de Davies-Bouldin
    
    print("Round {}/100 finished".format(ext_rnd + 1))
    
unique_dunn, counts_dunn = np.unique(k_by_dunn, return_counts=True)
unique_ch, counts_ch = np.unique(k_by_ch, return_counts=True)
unique_db, counts_db = np.unique(k_by_db, return_counts=True)

print("Best K per Round by Dunn Index:", dict(zip(unique_dunn, counts_dunn)))
print("Best K per Round by Calinski-Harabasz Index:", dict(zip(unique_ch, counts_ch)))
print("Best K per Round by Davies-Bouldin Index:", dict(zip(unique_db, counts_db)))

Round 1/100 finished
Round 2/100 finished
Round 3/100 finished
Round 4/100 finished
Round 5/100 finished
Round 6/100 finished
Round 7/100 finished
Round 8/100 finished
Round 9/100 finished
Round 10/100 finished
Round 11/100 finished
Round 12/100 finished
Round 13/100 finished
Round 14/100 finished
Round 15/100 finished
Round 16/100 finished
Round 17/100 finished
Round 18/100 finished
Round 19/100 finished
Round 20/100 finished
Round 21/100 finished
Round 22/100 finished
Round 23/100 finished
Round 24/100 finished
Round 25/100 finished
Round 26/100 finished
Round 27/100 finished
Round 28/100 finished
Round 29/100 finished
Round 30/100 finished
Round 31/100 finished
Round 32/100 finished
Round 33/100 finished
Round 34/100 finished
Round 35/100 finished
Round 36/100 finished
Round 37/100 finished
Round 38/100 finished
Round 39/100 finished
Round 40/100 finished
Round 41/100 finished
Round 42/100 finished
Round 43/100 finished
Round 44/100 finished
Round 45/100 finished
Round 46/100 finish

In [71]:
# Batch K-medians
def kmedians(data, proto, epochs):
    N, p = data.shape
    K = proto.shape[0]
    SSD = np.zeros(epochs)
    
    updt_proto = np.copy(proto)
    
    for epoch in range(epochs):
        dist = np.zeros((N, K))
        for t in range(N):
            dist[t, :] = np.sum(np.abs(data[t, :] - updt_proto), axis=1)
        
        cluster = np.argmin(dist, axis=1)
        SSD[epoch] = np.sum(np.min(dist, axis=1)**2)
    
        for k in range(K):
            I = np.where(cluster == k)[0]
            partition = data[I, :]
            updt_proto[k, :] = np.median(partition, axis=0)

    return updt_proto, SSD